Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [0]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [0]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # L2 regularizer
  regularizer = tf.nn.l2_loss(weights)
  loss = loss + beta * regularizer
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 48.601952
Minibatch accuracy: 7.8%
Validation accuracy: 9.6%
Minibatch loss at step 500: 0.986666
Minibatch accuracy: 82.0%
Validation accuracy: 79.5%
Minibatch loss at step 1000: 0.880977
Minibatch accuracy: 79.7%
Validation accuracy: 81.4%
Minibatch loss at step 1500: 0.622092
Minibatch accuracy: 85.9%
Validation accuracy: 80.5%
Minibatch loss at step 2000: 0.719007
Minibatch accuracy: 84.4%
Validation accuracy: 80.1%
Minibatch loss at step 2500: 0.685565
Minibatch accuracy: 81.2%
Validation accuracy: 80.3%
Minibatch loss at step 3000: 0.744972
Minibatch accuracy: 81.2%
Validation accuracy: 79.9%
Test accuracy: 87.9%


In [0]:
batch_size = 128
hidden_size = 1024

graph = tf.Graph()
with graph.as_default():
  
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size]))
  B1 = tf.Variable(tf.zeros([hidden_size]))
  
  W2 = tf.Variable(tf.truncated_normal([hidden_size, num_labels]))
  B2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation
  logits_1 = tf.matmul(tf_train_dataset, W1) + B1
  relu_1 = tf.nn.relu(logits_1)
  
  logits_2 = tf.matmul(relu_1, W2) + B2
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_2))
  
  # Regulization
  w1_regularizer = tf.nn.l2_loss(W1)
  w2_regularizer = tf.nn.l2_loss(W2)
  
  loss = loss + beta * (w1_regularizer + w2_regularizer)
  
  # Optimizer
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data
  train_prediction = tf.nn.softmax(logits_2)
  
  valid_y1 = tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + B1)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_y1, W2) + B2)
  
  test_y1 = tf.nn.relu(tf.matmul(tf_test_dataset, W1) + B1)
  test_prediction = tf.nn.softmax(tf.matmul(test_y1, W2) + B2)

In [8]:
num_step = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  
  for step in range(num_step):
    # Pick an offset within the training data, which has been randomized
    # Note: we could use better randomization across epochs
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    # Generate a minibatch
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch
    # The key of the dictionary is the placeholder node of the graph to be fed
    # and the value is the numpy array to feed to it
    feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step%500 == 0):
      print("Minibatch loss at %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy; %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at 0: 3442.979980
Minibatch accuracy: 10.2%
validation accuracy: 28.4%
Minibatch loss at 500: 21.340502
Minibatch accuracy: 82.0%
validation accuracy: 83.5%
Minibatch loss at 1000: 0.965351
Minibatch accuracy: 79.7%
validation accuracy: 82.9%
Minibatch loss at 1500: 0.635982
Minibatch accuracy: 87.5%
validation accuracy: 82.5%
Minibatch loss at 2000: 0.645032
Minibatch accuracy: 85.9%
validation accuracy: 83.0%
Minibatch loss at 2500: 0.645749
Minibatch accuracy: 85.9%
validation accuracy: 82.5%
Minibatch loss at 3000: 0.726680
Minibatch accuracy: 82.8%
validation accuracy: 82.6%
Test accuracy; 90.2%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [9]:
num_step = 3001

#train_dataset = train_dataset[:300, :]
#train_labels = train_labels[:300, :]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  
  for step in range(num_step):
    # Pick an offset within the training data, which has been randomized
    # Note: we could use better randomization across epochs
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    # Generate a minibatch
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch
    # The key of the dictionary is the placeholder node of the graph to be fed
    # and the value is the numpy array to feed to it
    feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step%500 == 0):
      print("Minibatch loss at %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy; %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at 0: 3484.671631
Minibatch accuracy: 12.5%
validation accuracy: 28.8%
Minibatch loss at 500: 21.329021
Minibatch accuracy: 83.6%
validation accuracy: 83.6%
Minibatch loss at 1000: 0.974193
Minibatch accuracy: 79.7%
validation accuracy: 82.9%
Minibatch loss at 1500: 0.635238
Minibatch accuracy: 87.5%
validation accuracy: 82.5%
Minibatch loss at 2000: 0.648695
Minibatch accuracy: 85.2%
validation accuracy: 82.8%
Minibatch loss at 2500: 0.646257
Minibatch accuracy: 88.3%
validation accuracy: 82.4%
Minibatch loss at 3000: 0.719782
Minibatch accuracy: 83.6%
validation accuracy: 82.6%
Test accuracy; 90.0%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [0]:
batch_size = 128
hidden_size = 1024

graph = tf.Graph()
with graph.as_default():
  
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size]))
  B1 = tf.Variable(tf.zeros([hidden_size]))
  
  W2 = tf.Variable(tf.truncated_normal([hidden_size, num_labels]))
  B2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation
  logits_1 = tf.matmul(tf_train_dataset, W1) + B1
  relu_1 = tf.nn.relu(logits_1)
  
  # Drop out for relu layer
  keep_prob = tf.placeholder("float")
  dropout_logits_1 = tf.nn.dropout(relu_1, keep_prob)
  
  logits_2 = tf.matmul(dropout_logits_1, W2) + B2
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_2))
  
  # Regulization
  w1_regularizer = tf.nn.l2_loss(W1)
  w2_regularizer = tf.nn.l2_loss(W2)
  
  loss = loss + beta * (w1_regularizer + w2_regularizer)
  
  # Optimizer
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data
  train_prediction = tf.nn.softmax(logits_2)
  
  valid_y1 = tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + B1)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_y1, W2) + B2)
  
  test_y1 = tf.nn.relu(tf.matmul(tf_test_dataset, W1) + B1)
  test_prediction = tf.nn.softmax(tf.matmul(test_y1, W2) + B2)

In [11]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 3557.66625977
Minibatch accuracy: 13.3
Validation accuracy: 30.8
Minibatch loss at step 500: 21.5077571869
Minibatch accuracy: 80.5
Validation accuracy: 82.9
Minibatch loss at step 1000: 1.01610136032
Minibatch accuracy: 78.9
Validation accuracy: 82.5
Minibatch loss at step 1500: 0.718991041183
Minibatch accuracy: 85.2
Validation accuracy: 82.1
Minibatch loss at step 2000: 0.750685751438
Minibatch accuracy: 82.8
Validation accuracy: 82.3
Minibatch loss at step 2500: 0.724893212318
Minibatch accuracy: 85.2
Validation accuracy: 82.1
Minibatch loss at step 3000: 0.830392837524
Minibatch accuracy: 82.0
Validation accuracy: 82.4
Test accuracy: 89.8


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [0]:
import math

In [0]:
batch_size = 128
beta = 0.001
hidden_size_1 = 1024
hidden_size_2 = 512
hidden_size_3 = 256
hidden_size_4 = 128
hidden_size_5 = 64

graph = tf.Graph()
with graph.as_default():
  
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size_1], stddev=math.sqrt(2.0/(image_size*image_size))))
  B1 = tf.Variable(tf.zeros([hidden_size_1]))
  
  W2 = tf.Variable(tf.truncated_normal([hidden_size_1, hidden_size_2], stddev=math.sqrt(2.0/hidden_size_1)))
  B2 = tf.Variable(tf.zeros([hidden_size_2]))
  
  W3 = tf.Variable(tf.truncated_normal([hidden_size_2, hidden_size_3], stddev=math.sqrt(2.0/hidden_size_2)))
  B3 = tf.Variable(tf.zeros([hidden_size_3]))
  
  W4 = tf.Variable(tf.truncated_normal([hidden_size_3, hidden_size_4], stddev=math.sqrt(2.0/hidden_size_3)))
  B4 = tf.Variable(tf.zeros([hidden_size_4]))
  
  W5 = tf.Variable(tf.truncated_normal([hidden_size_4, hidden_size_5], stddev=math.sqrt(2.0/hidden_size_4)))
  B5 = tf.Variable(tf.zeros([hidden_size_5]))
  
  W6 = tf.Variable(tf.truncated_normal([hidden_size_5, num_labels], stddev=math.sqrt(2.0/hidden_size_5)))
  B6 = tf.Variable(tf.zeros([num_labels]))
  
  # Drop out for relu layer
  keep_prob = tf.placeholder("float")
  
  # Training computation
  logits_1 = tf.matmul(tf_train_dataset, W1) + B1
  relu_1 = tf.nn.relu(logits_1)
  dropout_logits_1 = tf.nn.dropout(relu_1, keep_prob)
  
  logits_2 = tf.matmul(dropout_logits_1, W2) + B2
  relu_2 = tf.nn.relu(logits_2)
  dropout_logits_2 = tf.nn.dropout(relu_2, keep_prob)
  
  logits_3 = tf.matmul(dropout_logits_2, W3) + B3
  relu_3 = tf.nn.relu(logits_3)
  dropout_logits_3 = tf.nn.dropout(relu_3, keep_prob)
  
  logits_4 = tf.matmul(dropout_logits_3, W4) + B4
  relu_4 = tf.nn.relu(logits_4)
  dropout_logits_4 = tf.nn.dropout(relu_4, keep_prob)
  
  logits_5 = tf.matmul(dropout_logits_4, W5) + B5
  relu_5 = tf.nn.relu(logits_5)
  dropout_logits_5 = tf.nn.dropout(relu_5, keep_prob)
  
  logits_6 = tf.matmul(dropout_logits_5, W6) + B6
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_6))
  
  # Regulization
  w1_regularizer = tf.nn.l2_loss(W1)
  w2_regularizer = tf.nn.l2_loss(W2)
  w3_regularizer = tf.nn.l2_loss(W3)
  w4_regularizer = tf.nn.l2_loss(W4)
  w5_regularizer = tf.nn.l2_loss(W5)
  w6_regularizer = tf.nn.l2_loss(W6)
  
  loss = loss + beta * (w1_regularizer + w2_regularizer + w3_regularizer + w4_regularizer + w5_regularizer + w6_regularizer)
  
  # Optimizer
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0,96)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data
  train_prediction = tf.nn.softmax(logits_6)
  
  valid_y1 = tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + B1)
  valid_y2 = tf.nn.relu(tf.matmul(valid_y1, W2) + B2)
  valid_y3 = tf.nn.relu(tf.matmul(valid_y2, W3) + B3)
  valid_y4 = tf.nn.relu(tf.matmul(valid_y3, W4) + B4)
  valid_y5 = tf.nn.relu(tf.matmul(valid_y4, W5) + B5)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_y5, W6) + B6)
  
  test_y1 = tf.nn.relu(tf.matmul(tf_test_dataset, W1) + B1)
  test_y2 = tf.nn.relu(tf.matmul(test_y1, W2) + B2)
  test_y3 = tf.nn.relu(tf.matmul(test_y2, W3) + B3)
  test_y4 = tf.nn.relu(tf.matmul(test_y3, W4) + B4)
  test_y5 = tf.nn.relu(tf.matmul(test_y4, W5) + B5)
  test_prediction = tf.nn.softmax(tf.matmul(test_y5, W6) + B6)

In [14]:
num_steps = 15000

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))

Initialized
Minibatch loss at step 0: 5.13097953796
Minibatch accuracy: 14.1
Validation accuracy: 10.5
Minibatch loss at step 500: 1.99946212769
Minibatch accuracy: 75.8
Validation accuracy: 81.3
Minibatch loss at step 1000: 1.39313006401
Minibatch accuracy: 81.2
Validation accuracy: 83.2
Minibatch loss at step 1500: 1.12482500076
Minibatch accuracy: 85.9
Validation accuracy: 83.3
Minibatch loss at step 2000: 0.968376278877
Minibatch accuracy: 87.5
Validation accuracy: 83.8
Minibatch loss at step 2500: 0.835720121861
Minibatch accuracy: 88.3
Validation accuracy: 84.6
Minibatch loss at step 3000: 0.917559564114
Minibatch accuracy: 85.2
Validation accuracy: 84.7
Minibatch loss at step 3500: 0.993162035942
Minibatch accuracy: 81.2
Validation accuracy: 82.3
Minibatch loss at step 4000: 0.824833750725
Minibatch accuracy: 85.2
Validation accuracy: 85.1
Minibatch loss at step 4500: 0.727626383305
Minibatch accuracy: 85.2
Validation accuracy: 85.0
Minibatch loss at step 5000: 0.898515820503
Mi